In [70]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API Key
from config import api_key

In [71]:
# Attempt to get Monthly data from API for USD vs. JPY
try:
    usd_yen = requests.get(f"https://www.alphavantage.co/query?function=FX_MONTHLY&from_symbol=USD&to_symbol=JPY&apikey={api_key}").json()
except:
    print("Something went wrong with the API request.")

In [72]:
# Convert to DataFrame
usdjpy_df = pd.DataFrame(usd_yen['Time Series FX (Monthly)'])
usdjpy_df = usdjpy_df.transpose()
usdjpy_df

,1. open,2. high,3. low,4. close
2024-11-08,152.28700,154.70200,151.34300,152.82600
2024-10-31,143.73599,153.86800,143.53200,151.98000
2024-09-30,146.38100,147.18800,139.57800,143.71001
2024-08-29,149.20100,149.76401,141.69701,144.89000
2024-07-31,161.51199,161.94200,148.50501,149.74400
...,...,...,...,...
2015-04-30,119.57600,120.83300,118.50700,119.45200
2015-03-31,119.84500,121.99600,118.33000,120.02200
2015-02-26,117.18500,120.40200,116.89600,119.37700
2015-01-29,119.89000,120.73600,115.86500,118.24800


In [73]:
# Clean Dataframe
renamed_df = usdjpy_df.rename(columns={"1. open":"Open", "2. high":"High", "3. low":"Low", "4. close":"Close"})
clean_df = renamed_df[["Open", "Close", "High", "Low"]].astype(float)
clean_df.head()


,Open,Close,High,Low
2024-11-08,152.28700,152.82600,154.70200,151.34300
2024-10-31,143.73599,151.98000,153.86800,143.53200
2024-09-30,146.38100,143.71001,147.18800,139.57800
2024-08-29,149.20100,144.89000,149.76401,141.69701
2024-07-31,161.51199,149.74400,161.94200,148.50501


In [74]:
# Summarize Open, Close, High, and Low of USD to JPY
start_time = list(usd_yen["Time Series FX (Monthly)"])[0]
end_time = list(usd_yen["Time Series FX (Monthly)"])[-1]
usdjpy_mean = clean_df["Open"].mean()
usdjpy_median = clean_df["Open"].median()
usdjpy_std = clean_df["Open"].std()
usdjpy_var = clean_df["Open"].var()
print(f"USD to JPY from {start_time} to {end_time} \nUSD to JPY Mean= {usdjpy_mean} \nUSD to JPY Median= {usdjpy_median} \nUSD to JPY STD Error= {usdjpy_std} \nUSD to JPY Variance= {usdjpy_var}")


USD to JPY from 2024-11-08 to 2014-12-31 
USD to JPY Mean= 119.70849924999999 
USD to JPY Median= 112.851 
USD to JPY STD Error= 15.261572240076962 
USD to JPY Variance= 232.91558723908776
